In [1]:
import pandas as pd

In [2]:
# loading the dataset and displaying the first rows to check if it is loaded
df = pd.read_csv('/kaggle/input/dataset-sfs/dataset-sfs.csv')
df.head()

,Phone Brand,Phone Model,Date Received,Date Ready,Phone Problem
0,Samsung,Galaxy S10,2022-05-15,2022-05-20,LCD Replacement
1,iPhone,12 Pro Max,2022-06-10,2022-06-15,Battery Replacement
2,Google,Pixel 5,2022-07-01,2022-07-05,Charging Port Repair
3,Huawei,P40,2022-08-02,2022-08-07,Battery Replacement
4,OnePlus,9T,2022-09-05,2022-09-10,Camera Repair


In [3]:
# check for missing values
df.isnull().sum()

Phone Brand      0
Phone Model      0
Date Received    0
Date Ready       0
Phone Problem    0
dtype: int64

In [4]:
# one-hot encoding on categorical variables 
df = pd.get_dummies(df, columns=['Phone Brand', 'Phone Model', 'Phone Problem'])

In [5]:
df.head()

,Date Received,Date Ready,Phone Brand_Google,Phone Brand_Huawei,Phone Brand_OnePlus,Phone Brand_Oppo,Phone Brand_Samsung,Phone Brand_Xiaomi,Phone Brand_iPhone,Phone Model_10,...,Phone Model_Y9 Prime 2019,Phone Model_Y9s,Phone Problem_Battery Replacement,Phone Problem_Camera Repair,Phone Problem_Charging Port Repair,Phone Problem_LCD Replacement,Phone Problem_Screen Repair,Phone Problem_Screen Replacement,Phone Problem_Speaker Replacement,Phone Problem_Water Damage Repair
0,2022-05-15,2022-05-20,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,2022-06-10,2022-06-15,False,False,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
2,2022-07-01,2022-07-05,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,2022-08-02,2022-08-07,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,2022-09-05,2022-09-10,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


In [6]:
# Convert 'Date Received' to datetime format
df['Date Received'] = pd.to_datetime(df['Date Received'])

# Extract additional features from 'Date Received'
df['Month Received'] = df['Date Received'].dt.month
df['Day of Week Received'] = df['Date Received'].dt.dayofweek


In [7]:
df.head()

,Date Received,Date Ready,Phone Brand_Google,Phone Brand_Huawei,Phone Brand_OnePlus,Phone Brand_Oppo,Phone Brand_Samsung,Phone Brand_Xiaomi,Phone Brand_iPhone,Phone Model_10,...,Phone Problem_Battery Replacement,Phone Problem_Camera Repair,Phone Problem_Charging Port Repair,Phone Problem_LCD Replacement,Phone Problem_Screen Repair,Phone Problem_Screen Replacement,Phone Problem_Speaker Replacement,Phone Problem_Water Damage Repair,Month Received,Day of Week Received
0,2022-05-15,2022-05-20,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,5,6
1,2022-06-10,2022-06-15,False,False,False,False,False,False,True,False,...,True,False,False,False,False,False,False,False,6,4
2,2022-07-01,2022-07-05,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,7,4
3,2022-08-02,2022-08-07,False,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,8,1
4,2022-09-05,2022-09-10,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,9,0


In [8]:
# Convert 'Date Ready' to datetime format
df['Date Ready'] = pd.to_datetime(df['Date Ready'])

# Calculate the repair duration in days
df['Repair Duration'] = (df['Date Ready'] - df['Date Received']).dt.days

In [9]:
df.head(100)

,Date Received,Date Ready,Phone Brand_Google,Phone Brand_Huawei,Phone Brand_OnePlus,Phone Brand_Oppo,Phone Brand_Samsung,Phone Brand_Xiaomi,Phone Brand_iPhone,Phone Model_10,...,Phone Problem_Camera Repair,Phone Problem_Charging Port Repair,Phone Problem_LCD Replacement,Phone Problem_Screen Repair,Phone Problem_Screen Replacement,Phone Problem_Speaker Replacement,Phone Problem_Water Damage Repair,Month Received,Day of Week Received,Repair Duration
0,2022-05-15,2022-05-20,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,5,6,5
1,2022-06-10,2022-06-15,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,6,4,5
2,2022-07-01,2022-07-05,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,7,4,4
3,2022-08-02,2022-08-07,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,8,1,5
4,2022-09-05,2022-09-10,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,9,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-12-25,2022-12-27,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,12,6,2
96,2022-12-27,2022-12-29,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,12,1,2
97,2022-12-29,2022-12-31,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,12,3,2
98,2022-12-31,2023-01-02,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,12,5,2
